# Unit Commitment (Electrical Power Generation)

## Problem Description

A set of power stations are committed to meeting the following electricity load demands over a 24 hour horizon:

Table 1

| Time Period | Demand (megawatts) |
| --- | --- |
| 12 pm to 6 am | 15000 |
| 6 am to 9 am | 30000 |
| 9 am to 3 pm | 25000 |
| 3 pm to 6 pm | 40000 |
| 6 pm to 12 pm | 27000 |

There are three types of generating units available: 12 of type 0, 10 of type 1 and five of type 2. Each generator has a minimum and a maximum power output level. There is an hourly cost of running each generator at minimum level. In addition, there is an extra hourly cost for each megawatt at which a unit is operated above the minimum level. A unit can be on or off, with a startup cost associated with transitioning from off to on. All this information is given in Tables 2 and 3.

In addition to meeting the estimated load demands there must be sufficient generators working at any time to make it possible to meet an increase in load of up to 15%. The increase woulf have to be accomplished by adjusting the output of generators already operatung within their permitted limits.

Table 2

| Type | Number | Minimum output (MW) | Maximum output (MW) |
| --- | --- | --- | --- |
| 0 | 12 |  850 | 2000 |
| 1 | 10 | 1250 | 1750 |
| 2 | 5 | 1500 | 4000 |


Table 3

| Type | Cost per hour (when on) | Cost per MWh above minimum | Startup cost |
| --- | --- | --- | --- |
| 0 | $\$1000$ | $\$2.00$ | $\$2000$ |
| 1 | $\$2600$ | $\$1.30$ | $\$1000$ |
| 2 | $\$3000$ | $\$3.00$ | $\$500$ |


## Objective

To determine which generators should be turned on or off at each hour within the 24-hour period to meet anticipated demand in order to minimize total cost.

By determining the most efficient combination of generators to meet the anticipated power demand over a 24-hour period, mathematical optimization directly contributes to minimizing operational costs while ensuring reliable power supply. This practical application underscores the significance of mathematical optimization in making strategic, cost-effective decisions in real-world energy management.

This model is example 15 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 270 – 271 and 325 – 326.



In [ ]:
%pip install gurobipy
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Parameters. input data of the model

ntypes = 3
nperiods = 5
maxstart0 = 5

generators = [12, 10, 5]
period_hours = [6, 3, 6, 3, 6]
demand = [15000, 30000, 25000, 40000, 27000]
min_load = [850, 1250, 1500]
max_load = [2000, 1750, 4000]
base_cost = [1000, 2600, 3000]
per_mw_cost = [2, 1.3, 3]
startup_cost = [2000, 1000, 500]

In [6]:
# model deployment.

# For each time period, there is an integer decision variable to capture the number of active generators of each type (ngen),
# an integer variable to capture the number of generators of each type we must start (nstart),
# and a continuous decision variable to capture the total power output for each generator type (output)

model = gp.Model('PowerGeneration')
ngen = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="ngen")
nstart = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="nstart")
output = model.addVars(ntypes, nperiods, vtype=GRB.CONTINUOUS, name="genoutput")

In [7]:
# constraints

# Generator count. to ensure the number of active generators does not exceed the number of generators

numgen = model.addConstrs(ngen[type, period] <= generators[type]
                         for type in range(ntypes) for period in range(nperiods))



In [8]:
# minimum and maximum power output per generator type.
# Total power output for a generator type depends on the number of generators of that type that are active.

min_output = model.addConstrs((output[type, period] >= min_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

max_output = model.addConstrs((output[type, period] <= max_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

In [9]:
# Meet demand
# Total output for each time period must meet predicted demand.

meet_demand = model.addConstrs(gp.quicksum(output[type, period] for type in range(ntypes)) >= demand[period]
                               for period in range(nperiods))

In [10]:
# sufficient reserve capacity

reserve = model.addConstrs(gp.quicksum(max_load[type]*ngen[type, period] for type in range(ntypes)) >= 1.15*demand[period]
                    for period in range(nperiods))

In [11]:
# Startup constraint
# Connecting the decision variables that capture active generators with the decision variables that count startups.

startup0 = model.addConstrs((ngen[type,0] <= maxstart0 + nstart[type,0])
                            for type in range(ntypes))

startup = model.addConstrs((ngen[type,period] <= ngen[type,period-1] + nstart[type,period])
                           for type in range(ntypes) for period in range(1,nperiods))

In [13]:
# Objective: minimize total cost

# Cost consists of three components: the cost for running active generation units,
# the cost to generate power beyond the minimum for each unit, and the cost to start up generation units.

active = gp.quicksum(base_cost[type]*period_hours[period]*ngen[type,period]
                    for type in range(ntypes) for period in range(nperiods))

per_mw = gp.quicksum(per_mw_cost[type]*period_hours[period]*(output[type,period] - min_load[type]*ngen[type,period])
                       for type in range(ntypes) for period in range(nperiods))

startup_obj = gp.quicksum(startup_cost[type]*nstart[type,period]
                         for type in range(ntypes) for period in range(nperiods))

model.setObjective(active + per_mw + startup_obj)

In [14]:
# we solve the optimization problem and use Gurobi to find the optimal solution.

model.write('junk.lp')
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 70 rows, 45 columns and 147 nonzeros
Model fingerprint: 0xd010fb19
Variable types: 15 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+04]
Presolve removed 16 rows and 1 columns
Presolve time: 0.00s
Presolved: 54 rows, 44 columns, 130 nonzeros
Variable types: 0 continuous, 44 integer (0 binary)
Found heuristic solution: objective 1123913.3000
Found heuristic solution: objective 1084300.0000

Root relaxation: objective 9.995143e+05, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 


## Analysis of results

A total cost of $\$1,002,540$ is required to meet the anticipated demand for electricity over the 24-hour time window.  

The following table shows the number of generators of each type that are active in each time period in the optimal solution:

In [15]:
rows = ["Type" + str(t) for t in range(ntypes)]
units = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        units.loc["Type"+str(t), p] = ngen[t,p].x
units

,0,1,2,3,4
Type0,12.0,12.0,12.0,12.0,12.0
Type1,3.0,8.0,8.0,9.0,9.0
Type2,-0.0,0.0,0.0,2.0,-0.0


The following shows the number of generators of each type that must be started in each time period to achieve the requirements (the model assumes that 5 are available at the beginning of the time horizon):

In [16]:
startups = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        startups.loc["Type"+str(t), p] = int(nstart[t,p].x)
startups

,0,1,2,3,4
Type0,7.0,0.0,0.0,0.0,0.0
Type1,0.0,5.0,0.0,1.0,0.0
Type2,0.0,0.0,0.0,2.0,0.0
